In [48]:
import numpy as np
import tensorflow as tf


In [28]:
keras = tf.keras
Input = keras.layers.Input
Conv2D = keras.layers.Conv2D
Flatten = keras.layers.Flatten
Dense = keras.layers.Dense
GlobalAveragePooling2D = keras.layers.GlobalAveragePooling2D
Model = keras.Model

In [67]:
train_data = np.load('./data/train_data.npy')
train_labels = np.load('./data/train_labels.npy')

In [69]:
train_data = train_data.reshape((*train_data.shape, 1))

#### Basic CNN Model

In [72]:
input_layer = Input(shape=(train_data.shape[1], train_data.shape[2], 1))
conv_1 = Conv2D(filters=24, kernel_size=3, activation='relu')(input_layer)
conv_2 = Conv2D(filters=36, kernel_size=3, activation='relu')(conv_1)
conv_3 = Conv2D(filters=48, kernel_size=2, activation='relu')(conv_2)
conv_4 = Conv2D(filters=60, kernel_size=2, activation='relu')(conv_3)
gap_1 = GlobalAveragePooling2D()(conv_4)
dense = Dense(30, activation='softmax')(gap_1)
model = Model(inputs=input_layer, outputs=dense)

In [73]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 128, 63, 1)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 126, 61, 24)       240       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 124, 59, 36)       7812      
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 123, 58, 48)       6960      
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 122, 57, 60)       11580     
_________________________________________________________________
global_average_pooling2d_6 ( (None, 60)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 30)                1830      
Total para

In [74]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [75]:
model.fit(train_data, train_labels, epochs=50, batch_size=512, verbose=1)

Epoch 1/50
51088/51088 [==============================] - 65s 1ms/step - loss: 3.3626 - acc: 0.0513
Epoch 2/50
51088/51088 [==============================] - 65s 1ms/step - loss: 3.2055 - acc: 0.0958
Epoch 3/50
14848/51088 [=======>......................] - ETA: 50s - loss: 3.1076 - acc: 0.1098

KeyboardInterrupt: 